In [1]:
import sys
sys.path.append("../..")
import os.path

from hydra import initialize, compose
from hydra.utils import instantiate

from pytorch_lightning.callbacks import ModelSummary
from pytorch_lightning import Trainer

import torch

In [2]:
def load_model(
        model_checkpoint: str,
) -> torch.nn.Module:
    model_dir = os.path.dirname(model_checkpoint)
    with initialize(config_path=os.path.join(model_dir, 'hydra')):
        cfg = compose('config.yaml')
    if "model" in cfg.keys():
        cfg["model"]["_target_"] = cfg["model"]["_target_"].replace(
            ".model.", ".network."
        )
        cfg["model"]["backbone"]["_target_"] = cfg["model"]["backbone"]["_target_"].replace(
            ".model.", ".network."
        )
        cfg["model"]["backbone"]["cartesian_weights_path"] = "../../" + cfg["model"]["backbone"]["cartesian_weights_path"]
        model = instantiate(
            cfg.model,
            optimizer_config=cfg.optimizer,
            _recursive_=False
        )
    else:
        cfg["network"]["backbone"]["cartesian_weights_path"] = "../../" + cfg["network"]["backbone"]["cartesian_weights_path"]
        model = instantiate(
            cfg.network,
            optimizer_config=cfg.optimizer,
            _recursive_=False
        )    
    state_dict = torch.load(model_checkpoint, map_location=torch.device("cpu"))
    model.load_state_dict(state_dict["state_dict"], strict=False)
    model = model.eval().cpu()
    model = model.requires_grad_(False)
    return model

In [3]:
network = load_model("../../data/models_jeanzay/conv_x1/9/last.ckpt")

/tmp/ipykernel_989112/4017227567.py:5: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with initialize(config_path=os.path.join(model_dir, 'hydra')):


In [4]:
summary = ModelSummary(max_depth=5)
trainer = Trainer()

/cerea_raid/users/finnt/usr/miniconda3/envs/subsinn/lib/python3.9/site-packages/pytorch_lightning/loops/utilities.py:91: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/cerea_raid/users/finnt/usr/miniconda3/envs/subsinn/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1812: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(


In [5]:
summary.on_fit_start(trainer, network)


   | Name                          | Type               | Params
----------------------------------------------------------------------
0  | loss_func                     | LaplaceNLLLoss     | 9     
1  | fixed_laplace                 | LaplaceNLLLoss     | 9     
2  | fixed_gaussian                | GaussianNLLLoss    | 9     
3  | best_metric                   | MinMetric          | 0     
4  | backbone                      | ConvBackbone       | 46.5 K
5  | backbone.to_cartesian         | ToCartesianLayer   | 0     
6  | backbone.from_cartesian       | FromCartesianLayer | 0     
7  | backbone.network              | Sequential         | 46.5 K
8  | backbone.network.0            | MultiScaleConv2D   | 46.2 K
9  | backbone.network.0.small_conv | Conv2d             | 23.2 K
10 | backbone.network.0.large_conv | Conv2d             | 23.0 K
11 | backbone.network.1            | BatchNorm2d        | 256   
12 | backbone.network.2            | ReLU               | 0     
13 | head_node    

In [6]:
print(sum(p.numel() for p in network.backbone.network.parameters()))

46464


In [7]:
for l in network.backbone.network:
    print(sum(p.numel() for p in l.parameters()))

46208
256
0
